In [133]:
import sys
sys.path.insert(0,'../models/')
sys.path.insert(0,'../datasets/')
sys.path.insert(0,'..')

import pandas as pd
import numpy as np
import json
from subprocess import Popen, PIPE, STDOUT
import re
from collections import defaultdict
from utils.info import get_db_bounds
from datasets import propbankbr_arg2se, propbankbr_iob2arg

import tensorflow as tf
# import tqdm
from models import PropbankEncoder
import config 

INPUT_DIR = '../datasets/binaries/1.0/'
PROPBANK_WAN50_PATH = '{:}wan50/deep_wan50.pickle'.format(INPUT_DIR)
PEARL_SRLEVAL_PATH = '../srlconll04/srl-eval.pl'

# SPN Chunker

Uma mente insana realiza um experimento *lúcido* com o dataset de chunking da conll e seu script de avaliação :)

## 1.1 Carregar dados

In [134]:
dfgs = pd.read_csv('../datasets/csvs/1.0/gs.csv', index_col=0, sep=',', encoding='utf-8')
column_files = [
    '../datasets/csvs/1.0/column_chunks/chunks.csv',
    '../datasets/csvs/1.0/column_predmarker/predicate_marker.csv',
    '../datasets/csvs/1.0/column_shifts_ctx_p/form.csv',
    '../datasets/csvs/1.0/column_shifts_ctx_p/gpos.csv',
    '../datasets/csvs/1.0/column_shifts_ctx_p/lemma.csv',
    '../datasets/csvs/1.0/column_iob/iob.csv',
    '../datasets/csvs/1.0/column_t/t.csv'
]

for col_f in column_files:
    _df = pd.read_csv(col_f, index_col=0, encoding='utf-8')
    dfgs = pd.concat((dfgs, _df), axis=1)

DISPLAY_COLUMNS = ['ID', 'P', 'FORM', 'GPOS', 'MARKER', 'ARG', 'T', 'IOB',
                   'CHUNK_ID', 'CHUNK_START', 'CHUNK_FINISH', 'CHUNK_LEN', 'CHUNK_CANDIDATE_ID']            
dfgs[DISPLAY_COLUMNS].head(33)  

,ID,P,FORM,GPOS,MARKER,ARG,T,IOB,CHUNK_ID,CHUNK_START,CHUNK_FINISH,CHUNK_LEN,CHUNK_CANDIDATE_ID
INDEX,,,,,,,,,,,,,
0,1,1,Brasília,PROP,0,*,*,O,1,0,1,1,0
1,2,1,Pesquisa_Datafolha,N,0,(A0*,A0,B-A0,2,1,4,3,35
2,3,1,publicada,V-PCP,0,*,A0,I-A0,2,1,4,3,35
3,4,1,hoje,ADV,0,*),A0,I-A0,2,1,4,3,35
4,5,1,revela,V-FIN,1,(V*),V,B-V,3,4,5,1,126
5,6,1,um,ART,1,(A1*,A1,B-A1,4,5,32,27,181
6,7,1,dado,N,1,*,A1,I-A1,4,5,32,27,181
7,8,1,supreendente,ADJ,1,*,A1,I-A1,4,5,32,27,181
8,9,1,:,PU,1,*,A1,I-A1,4,5,32,27,181


## 1.2 Load Encodings

Propbank Encoder holds an indexed version of propbank dataset an answers to FOUR different dataformats: 
* CAT: this is the raw categorical data.
* EMB: tokens are embedding using GloVe embeddings.
* HOT: onehot encoding of the words and tokens.
* IDX: dense indexed representations.

In [135]:
# LOAD ENCODER
propbank_encoder = PropbankEncoder.recover(PROPBANK_WAN50_PATH)
db = propbank_encoder.db
lex2idx = propbank_encoder.lex2idx
idx2lex = propbank_encoder.idx2lex

# FOR TEXTUAL DATA ONLY
tok2idx = propbank_encoder.tok2idx
lex2tok = propbank_encoder.lex2tok
idx2word = propbank_encoder.idx2word

#Numpyfy embeddings
embeddings = propbank_encoder.embeddings

embeddings = np.concatenate([np.array(embs).reshape((1,50))
                             for embs in embeddings], axis=0)
n_targets = len(lex2idx['IOB'])

In [136]:
print(
'''Overall:
  \tattributes:{:}\trecords:{:}\tvocab:{:}\tpropositions:{:}'''
    .format(len(db), len(db['ARG'].keys()), 
            len(set([form for _, form in db['FORM'].items()])),
            len(set([p for _, p in db['P'].items()]))))

Overall:
  	attributes:43	records:138378	vocab:13206	propositions:5776


# 2. Data Wrangling

## 2.1 Helpful  function

In [137]:
def filter_type(ds_type, db):
    '''Filters only records from train dataset
    '''
    lb, ub = get_db_bounds(ds_type)

    sel_keys_ = {key_ for key_, prop_ in db['P'].items() if prop_ >= lb and prop_ < ub}

    return {
                attr_:{ idx_: i_
                        for idx_, i_ in dict_.items() if idx_ in sel_keys_
                      }        
                for attr_, dict_  in db.items()
            }

def make_propositions_dict(db):
    '''Reindex db by propositions creating a nested dict in which the
        outer key is the proposition        
    '''
    
    triple_list = []
    prev_idx = min(db['P'].keys())
    prev_prop = min(db['P'].values()) - 1 # Always enter first time
    first = True
    for idx, prop in db['P'].items():        
        if prev_prop != prop:
            if not first:
                ub = prev_idx
                triple_list.append((lb, ub, prev_prop))
            lb = idx
            first = False
        prev_prop = prop
        prev_idx = idx
    triple_list.append((lb, prev_idx, prev_prop))
            

        
    prop_set = set(db['P'].values())
    return { prop_:
                    {
                        attr_:{ idx_: dict_[idx_]
                                for idx_ in range(lb_, ub_ + 1, 1)
                          }        
                        for attr_, dict_ in db.items() if attr_ not in ('P',)
                    }
             for lb_, ub_, prop_ in  triple_list
            }, {prop_: ub_ - lb_ + 1 for lb_, ub_, prop_ in  triple_list}   


def numpfy_propositions_dict(prop_dict, proplen_dict):
    '''Converts inner dict examples into numpy arrays
    '''
    prop_dict_ = defaultdict(dict)    
    for prop, columns_dict in prop_dict.items():
        len_ = proplen_dict[prop]
        shape_ = (len_, 1)
        for column, values_dict in columns_dict.items():
            tuple_list = [idx_value 
                          for idx_value in values_dict.items()]

            tuple_list = sorted(tuple_list, key=lambda x: x[0])            
            # Converts lexicon (raw/indexed) into token (embedded/indexed)
            if (('FORM' in column) or ('LEMMA' in column)):
                values_list = [tok2idx[lex2tok[idx2word[tuple_[1]]]]                
                                   for tuple_ in tuple_list]
            else:
                values_list = [tuple_[1] for tuple_ in tuple_list]

            prop_dict_[prop][column]  = np.array(values_list).reshape(shape_)

    return prop_dict_        


## 2.2 Data segmentation

In [138]:
dbtrain = filter_type('train', db)
dbvalid = filter_type('valid', db)
dbtest = filter_type('test', db)

In [139]:
print(
'''Train:
  \tattributes:{:}\trecords:{:07d}\tvocab:{:07d}\tpropositions:{:}'''
    .format(len(dbtrain), 
            len(dbtrain['ARG'].keys()), 
            len(set([form for _, form in dbtrain['FORM'].items()])),
            len(set([p for _, p in dbtrain['P'].items()]))))

print(
'''Valid:
  \tattributes:{:}\trecords:{:07d}\tvocab:{:07d}\tpropositions:{:}'''
    .format(len(dbvalid),
            len(dbvalid['ARG'].keys()), 
            len(set([form for _, form in dbvalid['FORM'].items()])),
            len(set([p for _, p in dbvalid['P'].items()]))))

print(
'''Test:
  \tattributes:{:}\trecords:{:07d}\tvocab:{:07d}\tpropositions:{:}'''
    .format(len(dbtest),
            len(dbtest['ARG'].keys()), 
            len(set([form for _, form in dbtest['FORM'].items()])),
            len(set([p for _, p in dbtest['P'].items()]))))

print(
'''Overall:
  \tattributes:{:}\trecords:{:07d}\tvocab:{:07d}\tpropositions:{:}'''
    .format(len(dbtrain) + len(dbvalid) + len(dbtest), 
            len(set(list(dbtrain['ARG'].keys()) + list(dbvalid['ARG'].keys()) + list(dbtest['ARG'].keys()))), 
            len(set([form for _, form in dbtrain['FORM'].items()] +
                    [form for _, form in dbvalid['FORM'].items()] +
                    [form for _, form in dbtest['FORM'].items()])),
             len(set([form for _, form in dbtrain['P'].items()] +
                    [form for _, form in dbvalid['P'].items()] +
                    [form for _, form in dbtest['P'].items()]))))


Train:
  	attributes:43	records:0128258	vocab:0012469	propositions:5295
Valid:
  	attributes:43	records:0004893	vocab:0001180	propositions:239
Test:
  	attributes:43	records:0005123	vocab:0001139	propositions:239
Overall:
  	attributes:129	records:0138274	vocab:0013196	propositions:5773


## 2.3 Nested proposition

In [140]:
dbtrain, lentrain = make_propositions_dict(dbtrain)
dbvalid, lenvalid = make_propositions_dict(dbvalid)
dbtest, lentest = make_propositions_dict(dbtest)

In [141]:
print(
'''Train:
  \tattributes:{:}\trecords:{:07d}\tvocab:{:07d}\tpropositions:{:}'''
    .format(len(dbtrain[1]) + 1, 
            sum([len(d['ARG']) for p, d in dbtrain.items()]), 
            len(set([v                    
                     for p, d in dbtrain.items()
                     for v in d['FORM'].values()])),
            len(lentrain)))

print(
'''Valid:
  \tattributes:{:}\trecords:{:07d}\tvocab:{:07d}\tpropositions:{:}'''
    .format(len(dbvalid[min(dbvalid)]) + 1,
            sum([len(d['ARG']) for p, d in dbvalid.items()]), 
            len(set([v
                     for p, d in dbvalid.items()
                     for v in d['FORM'].values()])),
            len(lenvalid)))

print(
'''Test:
  \tattributes:{:}\trecords:{:07d}\tvocab:{:07d}\tpropositions:{:}'''
    .format(len(dbtest[min(dbtest)]) + 1,
            sum([len(d['ARG']) for p, d in dbtest.items()]), 
            len(set([v
                     for p, d in dbtest.items()
                     for v in d['FORM'].values()])),
            len(lentest)))

print(
'''Overall:
  \tattributes:{:}\trecords:{:07d}\tvocab:{:07d}\tpropositions:{:}'''
    .format(len(dbtrain[1]) + 1, 
            sum([len(d['ARG']) for p, d in dbtrain.items()] +
                [len(d['ARG']) for p, d in dbvalid.items()] +
                [len(d['ARG']) for p, d in dbtest.items()]), 
            len(set([v                    
                     for p, d in dbtrain.items()
                     for v in d['FORM'].values()] +
                    [v                    
                     for p, d in dbvalid.items()
                     for v in d['FORM'].values()] +
                    [v
                     for p, d in dbtest.items()  
                     for v in d['FORM'].values()])),
             len(lentrain) + len(lenvalid) + len(lentest)))


Train:
  	attributes:43	records:0128258	vocab:0012469	propositions:5295
Valid:
  	attributes:43	records:0004893	vocab:0001180	propositions:239
Test:
  	attributes:43	records:0005123	vocab:0001139	propositions:239
Overall:
  	attributes:43	records:0138274	vocab:0013196	propositions:5773


## 2.4 Numpfy

In [142]:
dbtrain = numpfy_propositions_dict(dbtrain, lentrain)
dbvalid = numpfy_propositions_dict(dbvalid, lenvalid)
dbtest = numpfy_propositions_dict(dbtest, lentest)

## 3. Data I/O

In [143]:
def get_inputs(db1, propid):
    '''Generate inputs
    '''
    propdb = db1[propid] # nested dict of columns and idx value
    try:
        if 'CHUNK_SPACE' not in propdb:
            proplen = len(propdb['ID'])
            propdb['CHUNK_SPACE'] = generate_chunk_space(proplen)

        # Replaces word with tokens
        word    = propdb['FORM']
        ctx_p_left  = propdb['FORM_CTX_P-1']
        ctx_p0  = propdb['FORM_CTX_P+0']
        ctx_p_right  = propdb['FORM_CTX_P+1']

        marker  = propdb['MARKER']
        pos     = propdb['GPOS']
        chunk_type  = propdb['T']
        chunk_start, chunk_finish = propdb['CHUNK_SPACE']
    except KeyError:
        import code; code.interact(local=dict(globals(), **locals()))

    return word, ctx_p_left, ctx_p0, ctx_p_right, marker, pos, chunk_type, chunk_start, chunk_finish
            
def generate_chunk_space(n):
    '''Generates all possible spaces for chunks
    '''
    start_list = []
    end_list = []
    for i in range(n):
        for j in range(i,n,1):
            start_list.append(i)
            end_list.append(j+1)
    shape_ = (len(start_list), 1)
    start_ = np.array(start_list).reshape(shape_)
    finish_ = np.array(end_list).reshape(shape_)
    return start_, finish_
            

def get_outputs(db1, propid, n_targets):
    ''' Generate outputs
    '''
    return db1[propid]['IOB']

In [144]:
%%timeit
propid = 1120
word, ctx_p_left, ctx_p0, ctx_p_right, marker, pos, chunk_type, chunk_start, chunk_finish = get_inputs(dbtrain, propid)
targets = get_outputs(dbtrain, propid, n_targets)


795 ns ± 5.46 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [145]:
propid = 1120
labels = get_outputs(dbtrain, propid, n_targets)
print(lex2idx['IOB'])
print(labels)

OrderedDict([('B-A0', 0), ('B-A1', 1), ('B-A2', 2), ('B-A3', 3), ('B-A4', 4), ('B-AM-ADV', 5), ('B-AM-CAU', 6), ('B-AM-DIR', 7), ('B-AM-DIS', 8), ('B-AM-EXT', 9), ('B-AM-LOC', 10), ('B-AM-MNR', 11), ('B-AM-NEG', 12), ('B-AM-PNC', 13), ('B-AM-PRD', 14), ('B-AM-REC', 15), ('B-AM-TMP', 16), ('B-V', 17), ('I-A0', 18), ('I-A1', 19), ('I-A2', 20), ('I-A3', 21), ('I-A4', 22), ('I-AM-ADV', 23), ('I-AM-CAU', 24), ('I-AM-DIR', 25), ('I-AM-DIS', 26), ('I-AM-EXT', 27), ('I-AM-LOC', 28), ('I-AM-MNR', 29), ('I-AM-NEG', 30), ('I-AM-PNC', 31), ('I-AM-PRD', 32), ('I-AM-REC', 33), ('I-AM-TMP', 34), ('I-V', 35), ('O', 36)])
[[36]
 [36]
 [36]
 [36]
 [36]
 [36]
 [36]
 [17]
 [10]
 [28]
 [28]
 [28]
 [28]
 [28]
 [36]
 [36]
 [36]
 [36]]


In [146]:
df = dfgs[dfgs['P'] == 1120]
df[DISPLAY_COLUMNS].head(18)

,ID,P,FORM,GPOS,MARKER,ARG,T,IOB,CHUNK_ID,CHUNK_START,CHUNK_FINISH,CHUNK_LEN,CHUNK_CANDIDATE_ID
INDEX,,,,,,,,,,,,,
27125,1,1120,Devo,V-FIN,0,*,*,O,5999,0,7,7,6
27126,2,1120,fingir,V-INF,0,*,*,O,5999,0,7,7,6
27127,3,1120,que,CONJ-S,0,*,*,O,5999,0,7,7,6
27128,4,1120,enxergo,V-FIN,0,*,*,O,5999,0,7,7,6
27129,5,1120,a,ART,0,*,*,O,5999,0,7,7,6
27130,6,1120,figura,N,0,*,*,O,5999,0,7,7,6
27131,7,1120,ou,CONJ-C,0,*,*,O,5999,0,7,7,6
27132,8,1120,sento,V-FIN,1,(V*),V,B-V,6000,7,8,1,105
27133,9,1120,a,PRP,1,(AM-LOC*,AM-LOC,B-AM-LOC,6001,8,14,6,121


## 4. Viterbi Functions

In [147]:
def longest_path(t_out, t_edge_scores, n_tags):
    
    def step(prev, et):
        # last computed scores and last computed transitions
        prev_scores, prev_selections = prev
        
        current_scores = tf.transpose(prev_scores + et)

        best_scores = tf.reduce_max(current_scores, axis=0)
        best_options = tf.argmax(current_scores, axis=0)

        return best_scores, best_options
    
    score_matrix, selection_matrix = tf.scan(
        fn=step,
        elems=t_edge_scores,
        initializer=(tf.zeros(n_tags), tf.to_int64(tf.zeros(n_tags))),
    )
    
    return score_matrix, selection_matrix

In [148]:
def retrieve_path(selection_matrix, last_tag):
    
    def step(prev, t):
        selection_matrix, prev_best = prev

        current = selection_matrix[t][prev_best]

        return selection_matrix, current

    m = tf.shape(selection_matrix)[0]
    _, rev_path = tf.scan(
        fn = step,
        elems=m-1-tf.range(m),
        initializer=(selection_matrix, last_tag)

    )

    best_path = tf.concat((tf.reverse(rev_path,axis=[0]),[last_tag]),axis=0)
    return best_path

In [149]:
def path_score(t_edge_scores, path, n):
    
    def step(prev, t):
        edge_scores, path, prev_score = prev
        
        transition_score = edge_scores[t]
        
        p_t = path[t]
        p_tp1 = path[t+1]

        current_score = transition_score[p_tp1,p_t] + prev_score

        return edge_scores, path, current_score

    _, _, path_score = tf.scan(
    fn = step,
    elems=tf.range(n-1),
    initializer=(t_edge_scores, path, tf.zeros(1))
    )
    return path_score[-1]


## 5. Build the computation graph

In [150]:
import tensorflow as tf
import numpy as np
import json
import time

In [151]:
hparams = {
    'hidden_features':200,
    'state_size':200,
    'learning_rate':0.001,
    'spn_layer':True
}

In [152]:
def make_graph(hparams):

    # Determine parameters for the computation graph
#     vocab_size = len(set(lex2tok.values()))
#     capt_size = model_meta['capt_size']
    pos_size = len(lex2idx['GPOS'])
#     char_size = model_meta['char_size']
    ck_size = len(lex2idx['IOB'])
    embed_size = embeddings.shape[1]
#     capt_embed_size = model_meta['capt_embed_size']
#     char_embed_size = model_meta['char_embed_size']
#     char_hidden = model_meta['char_hidden_features']
    pos_embed_size = len(idx2lex['GPOS'])

    hidden_features = hparams['hidden_features']
    state_size = hparams['state_size']
    lr = hparams['learning_rate']
    spn_layer = hparams['spn_layer']

    tf.reset_default_graph()
# word, ctx_p_left, ctx_p0, ctx_p_right, marker, pos, 

    t_x_words = tf.placeholder(tf.int32,(None,1))   # ids form
    t_x_ctx_p_left = tf.placeholder(tf.int32,(None,1))
    t_x_ctx_p0 = tf.placeholder(tf.int32,(None,1)) 
    t_x_ctx_p_right = tf.placeholder(tf.int32,(None,1))
    
    t_x_pos = tf.placeholder(tf.int32,(None,1))     # ids POS
    t_x_marker = tf.placeholder(tf.int32, (None,1))
    
    t_y_ck = tf.placeholder(tf.int32, shape=(None,)) # ids IOB-SRL
    
    
    t_inputs = [t_x_words, t_x_ctx_p_left, t_x_ctx_p0, t_x_ctx_p_right, t_x_marker, t_x_pos]
    t_targets = [t_y_ck]

    with tf.variable_scope('Feature_Vars'):
        t_W_embed = tf.Variable(embeddings.astype(np.float32))
#         t_W_char = tf.Variable(np.random.normal(0,0.1,(char_size+1, char_embed_size)).astype(np.float32))

        t_gamma = tf.Variable(np.random.normal(0,1.0, 1).astype(np.float32))
        t_beta = tf.Variable(np.random.normal(0,1.0, 1).astype(np.float32))

        dim = state_size
        if dim is None:
            dim = 2*hidden_features
        if spn_layer:
            t_W_tran = tf.Variable(np.random.normal(0,1.0/np.sqrt(3*dim),(3*dim, ck_size*ck_size)).astype(np.float32))
        else:
            t_W_ck = tf.Variable(np.random.normal(0,0.1,(dim, ck_size)).astype(np.float32))
        


    t_words = tf.gather_nd(t_W_embed, t_x_words)
    t_ctx_p_left = tf.gather_nd(t_W_embed, t_x_ctx_p_left)
    t_ctx_p0 = tf.gather_nd(t_W_embed, t_x_ctx_p0)
    t_ctx_p_right = tf.gather_nd(t_W_embed, t_x_ctx_p_right)
                                     
    t_x_pos_flat = tf.squeeze(t_x_pos, axis=1)

    t_pos = tf.to_float(tf.one_hot(indices= t_x_pos_flat, depth=pos_size, on_value=1,off_value=0))
    t_marker = tf.cast(t_x_marker, tf.float32)
    
    t_word_feats = tf.concat((t_words, t_ctx_p_left, t_ctx_p0, t_ctx_p_right, t_marker, t_pos), axis=1)
    print(t_word_feats.get_shape())

    t_sq_len = tf.shape(t_x_words)[0]
    print(t_sq_len)

    n_features = embed_size

    t_words_shp = tf.reshape(
        t_word_feats, (1,t_sq_len, pos_size + embed_size * 4 + 1)
    )

    cell_fw = tf.nn.rnn_cell.LSTMCell(num_units=hidden_features, state_is_tuple=True)
    cell_bw = tf.nn.rnn_cell.LSTMCell(num_units=hidden_features, state_is_tuple=True)

    with tf.variable_scope("Bilstm"):
        t_h1, t_last_states =tf.nn.bidirectional_dynamic_rnn(
            cell_fw=cell_fw,
            cell_bw=cell_bw,
            dtype=tf.float32,
            inputs=t_words_shp)

        t_hidden = tf.concat((t_h1[0][0],t_h1[1][0]),axis=1)

    if state_size is not None:
        t_lstmcell = tf.nn.rnn_cell.LSTMCell(num_units=state_size, state_is_tuple=True)
        t_hidden_shp = tf.reshape(t_hidden, (1,t_sq_len, 2*hidden_features))

        with tf.variable_scope('LSTM_last'):
            t_h2, t_last_states2 =tf.nn.dynamic_rnn(
                cell=t_lstmcell,
                dtype=tf.float32,
                sequence_length=[t_sq_len],
                inputs=t_hidden_shp)

        t_out = t_h2[0]
    else:
        t_out = t_hidden
        
    n = tf.shape(t_out)[0]
    
    t_outputs = []
    if spn_layer:
        t_edges = tf.concat((t_out[1:],t_out[:-1],t_out[1:]*t_out[:-1]),axis=1)
        t_edge_scores = tf.matmul(t_edges, t_W_tran)

        t_edge_scores = tf.reshape(t_edge_scores, ((n-1)*ck_size*ck_size,))

        # Batch Normalization
        t_es_mean = tf.reduce_mean(t_edge_scores)
        t_es_m2 = tf.reduce_mean(t_edge_scores**2)

        t_es_var = t_es_m2 - t_es_mean**2
        t_es_std = tf.sqrt(t_es_var + 1e-8)

        t_es_norm = (t_edge_scores - t_es_mean)/t_es_std

        t_es_renorm = t_gamma * t_es_norm + t_beta

        t_edge_scores = tf.reshape(t_es_renorm, (n-1,ck_size,ck_size))

        t_score_matrix, t_selection_matrix = longest_path(t_out, t_edge_scores,ck_size)

        t_best_score = tf.reduce_max(t_score_matrix[-1])
        t_last_tag = tf.argmax(t_score_matrix[-1])

        t_best_path = retrieve_path(t_selection_matrix, t_last_tag)

        t_correct_score = path_score(t_edge_scores, t_y_ck, n)

        t_cost = -t_correct_score

        # # gradiente descendente no custo do perceptron estruturado
        t_optimizer = tf.train.AdamOptimizer(learning_rate=lr)
        t_train = t_optimizer.minimize(t_cost)

        t_outputs.extend([t_score_matrix, t_best_path])
        
        # word, ctx_p_left, ctx_p0, ctx_p_right, marker, pos, _, _, _ = get_inputs(dbtrain, sample)
        def t_pred(sess, x_word, x_ctx_p_left, x_ctx_p0, x_ctx_p_right, x_marker, x_pos):
            result = sess.run(t_best_path, feed_dict={
                t_inputs[0]:x_word,
                t_inputs[1]:x_ctx_p_left,
                t_inputs[2]:x_ctx_p0,
                t_inputs[3]:x_ctx_p_right,                
                t_inputs[4]:x_marker,
                t_inputs[5]:x_pos
            })
            return result

        # word, ctx_p_left, ctx_p0, ctx_p_right, marker, pos, _, _, _ = get_inputs(dbtrain, sample)
        def my_t_train(sess, word, ctx_p_left, ctx_p0, ctx_p_right, marker, pos, y_ck):
            _, result = sess.run([t_train, t_cost], feed_dict={
                t_inputs[0]:x_word,
                t_inputs[1]:x_ctx_p_left,
                t_inputs[2]:x_ctx_p0,
                t_inputs[3]:x_ctx_p_right,                
                t_inputs[4]:x_marker,
                t_inputs[5]:x_pos,
                t_targets[0]:y_ck
            })
            return result
    else: #CRF
        t_ck_score = tf.matmul(t_out,t_W_ck)

        t_ck_score_ext = tf.expand_dims(t_ck_score, 0)
        t_y_ck_ext = tf.expand_dims(t_y_ck, 0)

        t_sequence_lengths = tf.shape(t_x_words)[0]
        t_sequence_lengths = tf.expand_dims(t_sequence_lengths,0)

        t_log_likelihood, t_transition_params = tf.contrib.crf.crf_log_likelihood(
            t_ck_score_ext, 
            t_y_ck_ext, 
            t_sequence_lengths)
        
        t_outputs.extend([t_ck_score, t_transition_params])
    
        t_cost = -t_log_likelihood

        # # gradiente descendente no custo do perceptron estruturado
        t_optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
        # optimizer = tf.train.GradientDescentOptimizer(0.003)
        t_train = t_optimizer.minimize(t_cost)

#         def t_pred(sess, inputs, x_words, x_capt, x_pos, x_char_matrix, x_char_lens):
        def t_pred(sess, x_word, x_ctx_p_left, x_ctx_p0, x_ctx_p_right, x_marker, x_pos):
            score, tparams = sess.run([t_ck_score,t_transition_params], feed_dict={
                t_inputs[0]:x_word,
                t_inputs[1]:x_ctx_p_left,
                t_inputs[2]:x_ctx_p0,
                t_inputs[3]:x_ctx_p_right,                
                t_inputs[4]:x_marker,
                t_inputs[5]:x_pos
            })

            return tf.contrib.crf.viterbi_decode(score=score,transition_params=tparams)[0][:-1]
        
        # word, ctx_p_left, ctx_p0, ctx_p_right, marker, pos, _, _, _ = get_inputs(dbtrain, sample)
        def my_t_train(sess, word, ctx_p_left, ctx_p0, ctx_p_right, marker, pos, y_ck):
            _, result = sess.run([t_train, t_cost], feed_dict={
                t_inputs[0]:x_word,
                t_inputs[1]:x_ctx_p_left,
                t_inputs[2]:x_ctx_p0,
                t_inputs[3]:x_ctx_p_right,                
                t_inputs[4]:x_marker,
                t_inputs[5]:x_pos,
                t_targets[0]:y_ck
            })
            return result
    
    return t_inputs, t_targets, t_outputs, my_t_train, t_pred


In [153]:
t_inputs, t_targets, t_outputs, t_train, t_pred = make_graph(hparams)

(?, 227)
Tensor("strided_slice:0", shape=(), dtype=int32)


In [154]:
sample = 146
# word, ctx_p_left, ctx_p0, ctx_p_right, marker, pos, chunk_type, chunk_start, chunk_finish
x_word, x_ctx_p_left, x_ctx_p0, x_ctx_p_right, x_marker, x_pos, _, _, _ = get_inputs(dbtrain, sample)

y_ck = get_outputs(dbtrain, sample, n_targets)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    o_ck = t_pred(sess, x_word, x_ctx_p_left, x_ctx_p0, x_ctx_p_right, x_marker, x_pos)
print(o_ck)
print(y_ck.shape)
print([idx2lex['IOB'][ck] for ck in list(o_ck)])

[31  6 34  6 34  6 34  6 34  6 34  6 34  6 34  6 34  6 34  6 34  6 34  6
 34  6 34  6 34  6 34  6 34  6 34  6 34  6 34  6 34  6 34  6 34  6 34  6
 34  6 34  6 34  6 34  6 34  6 34  6 34  6 34  6 34  6 34  6 34  6 34  6
 34]
(73, 1)
['I-AM-PNC', 'B-AM-CAU', 'I-AM-TMP', 'B-AM-CAU', 'I-AM-TMP', 'B-AM-CAU', 'I-AM-TMP', 'B-AM-CAU', 'I-AM-TMP', 'B-AM-CAU', 'I-AM-TMP', 'B-AM-CAU', 'I-AM-TMP', 'B-AM-CAU', 'I-AM-TMP', 'B-AM-CAU', 'I-AM-TMP', 'B-AM-CAU', 'I-AM-TMP', 'B-AM-CAU', 'I-AM-TMP', 'B-AM-CAU', 'I-AM-TMP', 'B-AM-CAU', 'I-AM-TMP', 'B-AM-CAU', 'I-AM-TMP', 'B-AM-CAU', 'I-AM-TMP', 'B-AM-CAU', 'I-AM-TMP', 'B-AM-CAU', 'I-AM-TMP', 'B-AM-CAU', 'I-AM-TMP', 'B-AM-CAU', 'I-AM-TMP', 'B-AM-CAU', 'I-AM-TMP', 'B-AM-CAU', 'I-AM-TMP', 'B-AM-CAU', 'I-AM-TMP', 'B-AM-CAU', 'I-AM-TMP', 'B-AM-CAU', 'I-AM-TMP', 'B-AM-CAU', 'I-AM-TMP', 'B-AM-CAU', 'I-AM-TMP', 'B-AM-CAU', 'I-AM-TMP', 'B-AM-CAU', 'I-AM-TMP', 'B-AM-CAU', 'I-AM-TMP', 'B-AM-CAU', 'I-AM-TMP', 'B-AM-CAU', 'I-AM-TMP', 'B-AM-CAU', 'I-AM-TMP', 'B-AM-CAU',

## 5.1 Overfit one proposition

In [155]:
sample =146
# word, ctx_p_left, ctx_p0, ctx_p_right, marker, pos, chunk_type, chunk_start, chunk_finish
# x_word, x_ctx_p_left, x_ctx_p0, x_ctx_p_right, x_marker, x_pos, _, _, _ = get_inputs(dbtrain, sample)
x_word, x_ctx_p_left, x_ctx_p0, x_ctx_p_right, x_marker, x_pos, _, _, _ = get_inputs(dbtrain, sample)
y_ck = get_outputs(dbtrain, sample, n_targets)
y_ck = np.squeeze(y_ck, axis=1)
sess = tf.Session()
sess.run(tf.global_variables_initializer())
for i in range(200):
    L = t_train(sess, x_word, x_ctx_p_left, x_ctx_p0, x_ctx_p_right, x_marker, x_pos, y_ck)
    if i % 100 == 0:
        print(L)
print(L)

[-59.620975]
[-1972.0256]
[-2385.631]


## 5.2 Evaluate one proposition

In [156]:
from subprocess import Popen, PIPE, STDOUT
import re

In [157]:
def tag_to_conll(sess, prop_dict, propid, idx2lex):
    gold_list = []
    eval_list = []
        
    x_word, x_ctx_p_left, x_ctx_p0, x_ctx_p_right, x_marker, x_pos, _, _, _ = get_inputs(prop_dict, propid)
    targets = get_outputs(prop_dict, propid, n_targets)
    predictions = t_pred(sess, x_word, x_ctx_p_left, x_ctx_p0, x_ctx_p_right, x_marker, x_pos)



    n_words = len(x_word)
    default_ck_list_ = [(i,i + 1, '*') for i in range(n_words)]
    
    pred_array = prop_dict[propid]['PRED']
    pred_array = pred_array.flatten()
    
    arg_array = prop_dict[propid]['ARG']
    arg_array = arg_array.flatten()
    
    pred_list = [idx2lex['PRED'][i] for i in pred_array.tolist()]
    gold_list_ = [idx2lex['ARG'][i] for i in arg_array.tolist()]
     
    gold_list += list(zip(pred_list, gold_list_))
    prop_list =  [propid for _ in range(len(gold_list))]
    eval_list = [idx2lex['IOB'][i] for i in predictions]
    
    #FIXME: removed #end of sentence token
    # now predictions are shorter then gold
    if len(eval_list) < len(prop_list):
        eval_list.append('O')

    eval_list = propbankbr_iob2arg(prop_list, eval_list)
    
#     ck_list_ = []     
#     for triple_ in sorted(chunk_ext, key= lambda x: x[0]):
#         lb, ub, ckid = triple_
#         # filters default value
#         default_ck_list_ = [
#             dck_
#             for dck_ in default_ck_list_ if dck_[0] < lb or dck_[1] > ub
#         ]
#         ck_list_.append((lb, ub, idx2lex['T'][ckid]))        

#     ck_list_ = default_ck_list_ + ck_list_ 

#     arg_list_ = []
#     for triple_ in sorted(ck_list_, key= lambda x: x[0]):
#         lb, ub, cktype = triple_
#         flat_list_ = [ cktype if i == lb else '*' for i in range(lb, ub) ]
            
#         if cktype != '*':
#             flat_list_[0] = '({:}*'.format(flat_list_[0])
#             flat_list_[-1] = '{:})'.format(flat_list_[-1])
#         arg_list_ += flat_list_
        
    eval_list = list(zip(pred_list, eval_list))
#     eval_list.append(None)
#     gold_list.append(None)
#     Change to use CoNLL2004
    gold_list = propbankbr_arg2se(gold_list)
    eval_list = propbankbr_arg2se(eval_list)

    return gold_list, eval_list

In [158]:
def evaluate(gold_list, eval_list, ds_type='train',verbose=True):
    gold_path = '{}_gold.props'.format(ds_type)    
    eval_path = '{}_eval.props'.format(ds_type)

    with open(gold_path, mode='w') as f:        
        for tuple_ in gold_list:
            if tuple_ is None:
                f.write('\n')
            else:
                f.write('{:}\t{:}\n'.format(*tuple_))

    with open(eval_path, mode='w') as f:        
        for tuple_ in eval_list:
            if tuple_ is None:
                f.write('\n')
            else:
                f.write('{:}\t{:}\n'.format(*tuple_))

    pipe = Popen(['perl',PEARL_SRLEVAL_PATH, gold_path, eval_path], stdout=PIPE, stderr=PIPE)

    txt, err = pipe.communicate()
    txt = txt.decode('UTF-8')
    err = err.decode('UTF-8')
    
    print(err)
    if verbose:
        print(txt)
        with open('{}.conll'.format(ds_type), mode='w') as f:
            f.write(txt)

    # overall is a summary from the list
    # is the seventh line
    lines_list = txt.split('\n')        
    
    # get the numbers from the row 
    overall_list = re.findall(r'[-+]?[0-9]*\.?[0-9]+.', lines_list[6])
    f1 = float(overall_list[-1])

    return f1



In [159]:
def evaluate_dataset(sess, prop_dict, idx2lex, ds_type='train'):
    gold_list = []
    eval_list = []
    first = True
    for pid in prop_dict:        
        g_list, e_list = tag_to_conll(sess, prop_dict, pid, idx2lex)

        if not first:
            gold_list.append(None)
            eval_list.append(None)
        else:
            first = False

        gold_list += g_list
        eval_list += e_list

    return evaluate(gold_list, eval_list, ds_type=ds_type, verbose=True)

In [160]:
gold_tags, eval_tags = tag_to_conll(sess, dbtrain, sample, idx2lex)
f1 = evaluate(gold_tags, eval_tags, 'prop_{:}'.format(sample), verbose=True)



Number of Sentences    :           1
Number of Propositions :           1
Percentage of perfect props : 100.00

              corr.  excess  missed    prec.    rec.      F1
------------------------------------------------------------
   Overall        4       0       0   100.00  100.00  100.00
----------
        A1        2       0       0   100.00  100.00  100.00
    AM-ADV        1       0       0   100.00  100.00  100.00
    AM-NEG        1       0       0   100.00  100.00  100.00
------------------------------------------------------------
         V        2       0       0   100.00  100.00  100.00
------------------------------------------------------------



In [161]:
f1_train = evaluate_dataset(sess, dbtrain, idx2lex, ds_type='train')





Number of Sentences    :        5295
Number of Propositions :        5295
Percentage of perfect props :   0.21

              corr.  excess  missed    prec.    rec.      F1
------------------------------------------------------------
   Overall      129   10616   12380     1.20    1.03    1.11
----------
        A0        0       0    2816     0.00    0.00    0.00
        A1       85    5706    4705     1.47    1.77    1.61
        A2        0       0    1027     0.00    0.00    0.00
        A3        0       0     106     0.00    0.00    0.00
        A4        0       0      66     0.00    0.00    0.00
    AM-ADV       17    1876     321     0.90    5.03    1.52
    AM-CAU        0       0     149     0.00    0.00    0.00
    AM-DIR        0       0      12     0.00    0.00    0.00
    AM-DIS        0       0     271     0.00    0.00    0.00
    AM-EXT        0       0      78     0.00    0.00    0.00
    AM-LOC        0       0     721     0.00    0.00    0.00
    AM-MNR        0   

## Treino Parcial

In [162]:
# model_meta = {
#     'vocab_size':len(id_to_word),
#     'capt_size':len(id_to_capt),
#     'pos_size':len(id_to_pos),
#     'char_size':len(id_to_char),
#     'ck_size':len(id_to_ck),
#     'embed_size':embedding_matrix.shape[1],
#     'capt_embed_size':10,
#     'char_hidden_features':30,
#     'char_embed_size':30,
#     'pos_embed_size':10,
#     'hidden_features':200,
#     'state_size':200,
#     'learning_rate':0.001,
#     'spn_layer':True
# }

# t_inputs, t_targets, t_outputs, t_train, t_pred = make_model(model_meta, embedding_matrix)

# sess = tf.Session()
# sess.run(tf.global_variables_initializer())

# n_epochs = 3

# indices = np.arange(len(train_sentences)//100)    
# n = len(indices)

# best_path = None
# best_f1 = 0

# for epoch in range(n_epochs):
#     j = 0
#     np.random.shuffle(indices)
#     for sid in indices:
#         j += 1
#         x_words, x_capt, x_chars, x_pos, x_char_matrix, x_lens = get_input(train_sentences[sid])
#         y_ck = get_output(train_sentences[sid])
#         L = t_train(sess,t_inputs,t_targets,x_words,x_capt,x_pos,x_char_matrix,x_lens,y_ck)
#         if j % (n//10) == 0:
#             print('{:.2f} %'.format(100*j/n))
        
#     print("Epoca ", (epoch+1))
#     dev_f1 = evaluate_conll(sess, df_ck, dev_sentences[:len(indices)],t_inputs,t_pred,verbose=False)
#     print("dev f1: ", dev_f1)
#     train_f1 = evaluate_conll(sess, df_ck, train_sentences[:len(indices)],t_inputs,t_pred,verbose=False)
#     print("train f1: ", train_f1)
    
#     if dev_f1 > best_f1:
#         best_f1 = dev_f1
# sess.close()

## Treino Total

In [163]:
import time
import datetime

train_sentences = range(*get_db_bounds('train'))
dev_sentences = range(*get_db_bounds('valid'))
test_sentences = range(*get_db_bounds('test'))

In [ ]:
lr_search = [0.0001]
hidden_features_search = [50,150,250]

for j in range(len(hidden_features_search)):
    for i in range(len(lr_search)):
        for s in range(2):
            lr = lr_search[i]
            hf = hidden_features_search[j]

            print(j, ' ', len(hidden_features_search))
#             model_meta = {
#                 'vocab_size':len(id_to_word),
#                 'capt_size':len(id_to_capt),
#                 'pos_size':len(id_to_pos),
#                 'char_size':len(id_to_char),
#                 'ck_size':len(id_to_ck),
#                 'embed_size':embedding_matrix.shape[1],
#                 'capt_embed_size':10,
#                 'char_embed_size':30,
#                 'pos_embed_size':10,
#                 'char_hidden_features':30,
#                 'hidden_features':hf,
#                 'state_size':hf,
#                 'learning_rate':lr,
#                 'spn_layer':s==0
#             }
            model_meta = {
                'hidden_features':hf,
                'state_size':hf,
                'learning_rate':lr,
                'spn_layer':s==0
            }
            print('Training Model', model_meta)
            
            last_layer = 'crf'
            if s == 0:
                last_layer = 'spn'

            n_epochs = 30
            model_name = 'bilstm_viterbi_h' + str(hf) + '_lr_' + str(lr) + '_' + last_layer
            model = make_graph(model_meta)


            t_inputs, t_targets, t_outputs, t_train, t_pred = model
            
            #NEW SESSION PER MODEL
            config = tf.ConfigProto()
            config.gpu_options.allow_growth = True
            sess = tf.Session(config=config)
            sess.run(tf.global_variables_initializer())

            indices = np.arange(1, len(train_sentences))
            n = len(indices)

            best_path = None
            best_f1 = 0

            save_dir = '../outputs/1.0/notebooks/spn/models/'
            exp_desc_dir = 'results/'
            saver = tf.train.Saver(max_to_keep=n_epochs*20)

            dev_f1_list = []
            train_f1_list = []
            consecutive_bad_dev = 0

            for epoch in range(n_epochs):
                start = time.time()
                np.random.shuffle(indices)
                it = 0
                for sid in indices:
                    it += 1
                    x_word, x_ctx_p_left, x_ctx_p0, x_ctx_p_right, x_marker, x_pos, _, _, _ = get_inputs(dbtrain, sid)
                    targets = get_outputs(dbtrain, sid, n_targets)
                    y_ck = np.squeeze(targets, axis=1)
#                     x_words, x_capt, x_chars, x_pos, x_char_matrix, x_lens = get_input(train_sentences[sid])
#                     y_ck = get_output(train_sentences[sid])
                    L = t_train(sess, x_word, x_ctx_p_left, x_ctx_p0, x_ctx_p_right, x_marker, x_pos, y_ck)
                print("Epoca ", (epoch+1))
#                 dev_f1 = evaluate_conll(sess, df_ck, dev_sentences,t_inputs,t_pred,verbose=False)
#                 gold_tags, eval_tags = tag_to_conll(sess, dbvalid, sample, idx2lex)
#                 f1 = evaluate(gold_tags, eval_tags, verbose=True)
                f1_valid = evaluate_dataset(sess, dbvalid, idx2lex, ds_type='valid')            

                print("valid f1: ", f1_valid)
#                 train_f1 = evaluate_conll(sess, df_ck, train_sentences,t_inputs,t_pred,verbose=False)
                f1_train = evaluate_dataset(sess, dbtrain, idx2lex, ds_type='train')            
                print("train f1: ", f1_train)
                dev_f1_list.append(f1_valid)
                train_f1_list.append(f1_train)
                end = time.time()
                print('Tempo por Epoca: ', (end-start), ' s')
                if f1_valid > best_f1:
                    best_f1 = f1_valid
                    timestamp = datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d_%H-%M-%S')
                    save_path = save_dir + model_name + '_' + timestamp
                    saver.save(sess, save_path)
                    print('## BEST MODEL saved at ', save_path)
                    consecutive_bad_dev = 0
                else:
                    consecutive_bad_dev += 1
                if consecutive_bad_dev == 5:
                    break
            timestamp = datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d_%H-%M-%S')
            exp_path = exp_desc_dir + model_name + timestamp + '.txt'
            with open(exp_path,'w') as f:
                exp_desc = {
                    'model':model_meta,
                    'f1_train':train_f1_list,
                    'f1_valid':dev_f1_list
                }
                json.dump(exp_desc, f)
                print('save results on ' + exp_path)

0   3
Training Model {'hidden_features': 50, 'state_size': 50, 'learning_rate': 0.0001, 'spn_layer': True}
(?, 227)
Tensor("strided_slice:0", shape=(), dtype=int32)
Epoca  1

Number of Sentences    :         239
Number of Propositions :         239
Percentage of perfect props :   0.42

              corr.  excess  missed    prec.    rec.      F1
------------------------------------------------------------
   Overall        0     135     558     0.00    0.00    0.00
----------
        A0        0       0     123     0.00    0.00    0.00
        A1        0     135     217     0.00    0.00    0.00
        A2        0       0      44     0.00    0.00    0.00
        A3        0       0       7     0.00    0.00    0.00
        A4        0       0       8     0.00    0.00    0.00
    AM-ADV        0       0      13     0.00    0.00    0.00
    AM-CAU        0       0       7     0.00    0.00    0.00
    AM-DIR        0       0       1     0.00    0.00    0.00
    AM-DIS        0       0    

## BEST MODEL saved at  ../outputs/1.0/notebooks/spn/models/bilstm_viterbi_h50_lr_0.0001_spn_2018-10-15_15-50-09
Epoca  4

Number of Sentences    :         239
Number of Propositions :         239
Percentage of perfect props :   5.44

              corr.  excess  missed    prec.    rec.      F1
------------------------------------------------------------
   Overall       56     134     502    29.47   10.04   14.97
----------
        A0       10       5     113    66.67    8.13   14.49
        A1       46     129     171    26.29   21.20   23.47
        A2        0       0      44     0.00    0.00    0.00
        A3        0       0       7     0.00    0.00    0.00
        A4        0       0       8     0.00    0.00    0.00
    AM-ADV        0       0      13     0.00    0.00    0.00
    AM-CAU        0       0       7     0.00    0.00    0.00
    AM-DIR        0       0       1     0.00    0.00    0.00
    AM-DIS        0       0      13     0.00    0.00    0.00
    AM-EXT        0   

## BEST MODEL saved at  ../outputs/1.0/notebooks/spn/models/bilstm_viterbi_h50_lr_0.0001_spn_2018-10-15_16-05-05
Epoca  7

Number of Sentences    :         239
Number of Propositions :         239
Percentage of perfect props :  10.46

              corr.  excess  missed    prec.    rec.      F1
------------------------------------------------------------
   Overall      100     232     458    30.12   17.92   22.47
----------
        A0       17      48     106    26.15   13.82   18.09
        A1       82     168     135    32.80   37.79   35.12
        A2        1      16      43     5.88    2.27    3.28
        A3        0       0       7     0.00    0.00    0.00
        A4        0       0       8     0.00    0.00    0.00
    AM-ADV        0       0      13     0.00    0.00    0.00
    AM-CAU        0       0       7     0.00    0.00    0.00
    AM-DIR        0       0       1     0.00    0.00    0.00
    AM-DIS        0       0      13     0.00    0.00    0.00
    AM-EXT        0   

## BEST MODEL saved at  ../outputs/1.0/notebooks/spn/models/bilstm_viterbi_h50_lr_0.0001_spn_2018-10-15_16-15-23
Epoca  10

Number of Sentences    :         239
Number of Propositions :         239
Percentage of perfect props :  15.06

              corr.  excess  missed    prec.    rec.      F1
------------------------------------------------------------
   Overall      133     228     425    36.84   23.84   28.94
----------
        A0       47      59      76    44.34   38.21   41.05
        A1       83     137     134    37.73   38.25   37.99
        A2        2      32      42     5.88    4.55    5.13
        A3        0       0       7     0.00    0.00    0.00
        A4        0       0       8     0.00    0.00    0.00
    AM-ADV        0       0      13     0.00    0.00    0.00
    AM-CAU        0       0       7     0.00    0.00    0.00
    AM-DIR        0       0       1     0.00    0.00    0.00
    AM-DIS        0       0      13     0.00    0.00    0.00
    AM-EXT        0  

## BEST MODEL saved at  ../outputs/1.0/notebooks/spn/models/bilstm_viterbi_h50_lr_0.0001_spn_2018-10-15_16-25-31
Epoca  13

Number of Sentences    :         239
Number of Propositions :         239
Percentage of perfect props :  15.06

              corr.  excess  missed    prec.    rec.      F1
------------------------------------------------------------
   Overall      151     223     407    40.37   27.06   32.40
----------
        A0       62      49      61    55.86   50.41   52.99
        A1       80     124     137    39.22   36.87   38.00
        A2        7      48      37    12.73   15.91   14.14
        A3        0       0       7     0.00    0.00    0.00
        A4        0       0       8     0.00    0.00    0.00
    AM-ADV        0       0      13     0.00    0.00    0.00
    AM-CAU        0       0       7     0.00    0.00    0.00
    AM-DIR        0       0       1     0.00    0.00    0.00
    AM-DIS        0       0      13     0.00    0.00    0.00
    AM-EXT        0  

## BEST MODEL saved at  ../outputs/1.0/notebooks/spn/models/bilstm_viterbi_h50_lr_0.0001_spn_2018-10-15_16-35-37
Epoca  16

Number of Sentences    :         239
Number of Propositions :         239
Percentage of perfect props :  19.67

              corr.  excess  missed    prec.    rec.      F1
------------------------------------------------------------
   Overall      176     219     382    44.56   31.54   36.94
----------
        A0       67      47      56    58.77   54.47   56.54
        A1       92     100     125    47.92   42.40   44.99
        A2       13      51      31    20.31   29.55   24.07
        A3        0       0       7     0.00    0.00    0.00
        A4        0       0       8     0.00    0.00    0.00
    AM-ADV        0       0      13     0.00    0.00    0.00
    AM-CAU        0       0       7     0.00    0.00    0.00
    AM-DIR        0       0       1     0.00    0.00    0.00
    AM-DIS        0       0      13     0.00    0.00    0.00
    AM-EXT        0  

Epoca  19

Number of Sentences    :         239
Number of Propositions :         239
Percentage of perfect props :  21.34

              corr.  excess  missed    prec.    rec.      F1
------------------------------------------------------------
   Overall      194     247     364    43.99   34.77   38.84
----------
        A0       76      63      47    54.68   61.79   58.02
        A1      100     105     117    48.78   46.08   47.39
        A2       11      45      33    19.64   25.00   22.00
        A3        0       0       7     0.00    0.00    0.00
        A4        0       0       8     0.00    0.00    0.00
    AM-ADV        0       0      13     0.00    0.00    0.00
    AM-CAU        0       0       7     0.00    0.00    0.00
    AM-DIR        0       0       1     0.00    0.00    0.00
    AM-DIS        0       0      13     0.00    0.00    0.00
    AM-EXT        0       0       2     0.00    0.00    0.00
    AM-LOC        1       5      29    16.67    3.33    5.56
    AM-MNR  

## BEST MODEL saved at  ../outputs/1.0/notebooks/spn/models/bilstm_viterbi_h50_lr_0.0001_spn_2018-10-15_16-55-52
Epoca  22

Number of Sentences    :         239
Number of Propositions :         239
Percentage of perfect props :  21.76

              corr.  excess  missed    prec.    rec.      F1
------------------------------------------------------------
   Overall      204     239     354    46.05   36.56   40.76
----------
        A0       80      55      43    59.26   65.04   62.02
        A1      103     103     114    50.00   47.47   48.70
        A2       10      36      34    21.74   22.73   22.22
        A3        0       0       7     0.00    0.00    0.00
        A4        0       0       8     0.00    0.00    0.00
    AM-ADV        0       0      13     0.00    0.00    0.00
    AM-CAU        0       0       7     0.00    0.00    0.00
    AM-DIR        0       0       1     0.00    0.00    0.00
    AM-DIS        0       0      13     0.00    0.00    0.00
    AM-EXT        0  

Epoca  25

Number of Sentences    :         239
Number of Propositions :         239
Percentage of perfect props :  22.18

              corr.  excess  missed    prec.    rec.      F1
------------------------------------------------------------
   Overall      208     249     350    45.51   37.28   40.99
----------
        A0       79      56      44    58.52   64.23   61.24
        A1      102     101     115    50.25   47.00   48.57
        A2       10      45      34    18.18   22.73   20.20
        A3        0       0       7     0.00    0.00    0.00
        A4        0       0       8     0.00    0.00    0.00
    AM-ADV        0       4      13     0.00    0.00    0.00
    AM-CAU        0       0       7     0.00    0.00    0.00
    AM-DIR        0       0       1     0.00    0.00    0.00
    AM-DIS        0       0      13     0.00    0.00    0.00
    AM-EXT        0       0       2     0.00    0.00    0.00
    AM-LOC        7      16      23    30.43   23.33   26.42
    AM-MNR  

## BEST MODEL saved at  ../outputs/1.0/notebooks/spn/models/bilstm_viterbi_h50_lr_0.0001_spn_2018-10-15_17-16-32
